# 汇总labelme标注信息-单个labelme标注文件

同济子豪兄 2023-4-23

## 导入工具包

In [54]:
import os
import json

import numpy as np
import pandas as pd
import cv2

from tqdm import tqdm

## 创建表格

In [55]:
df = pd.DataFrame()

## 读取单个labelme文件

In [56]:
labelme_path = '1_labelme.json'

In [57]:
with open(labelme_path, 'r', encoding='utf-8') as f:
    labelme = json.load(f)

In [58]:
imagePath = labelme['imagePath']
imageWidth = labelme['imageWidth']
imageHeight = labelme['imageHeight']

In [59]:
for each_ann in tqdm(labelme['shapes']): # 遍历每一个标注
    
    df_temp = {}
    
    # 图像信息
    df_temp['imagePath'] = imagePath
    df_temp['imageWidth'] = imageWidth
    df_temp['imageHeight'] = imageHeight
    
    if each_ann['shape_type'] == 'rectangle': # 筛选出框标注
        
        # 框的两点坐标
        bbox_keypoints = each_ann['points'] 
        bbox_keypoint_A_xy = bbox_keypoints[0]
        bbox_keypoint_B_xy = bbox_keypoints[1]
        # 左上角坐标
        bbox_top_left_x = int(min(bbox_keypoint_A_xy[0], bbox_keypoint_B_xy[0]))
        bbox_top_left_y = int(min(bbox_keypoint_A_xy[1], bbox_keypoint_B_xy[1]))
        # 右下角坐标
        bbox_bottom_right_x = int(max(bbox_keypoint_A_xy[0], bbox_keypoint_B_xy[0]))
        bbox_bottom_right_y = int(max(bbox_keypoint_A_xy[1], bbox_keypoint_B_xy[1]))
        
        # 标注信息
        df_temp['label_type'] = each_ann['shape_type']
        df_temp['label'] = each_ann['label'] 
        
        # 框坐标信息
        df_temp['bbox_top_left_x'] = bbox_top_left_x
        df_temp['bbox_top_left_y'] = bbox_top_left_y
        df_temp['bbox_bottom_right_x'] = bbox_bottom_right_x
        df_temp['bbox_bottom_right_y'] = bbox_bottom_right_y
        df_temp['bbox_width_pix'] = bbox_bottom_right_x - bbox_top_left_x
        df_temp['bbox_height_pix'] = bbox_bottom_right_y - bbox_top_left_y
        df_temp['bbox_width_norm'] = df_temp['bbox_width_pix'] / df_temp['imageWidth']
        df_temp['bbox_height_norm'] = df_temp['bbox_height_pix'] / df_temp['imageHeight']
        df_temp['bbox_center_x_pix'] = (bbox_top_left_x + bbox_bottom_right_x) / 2
        df_temp['bbox_center_y_pix'] = (bbox_top_left_y + bbox_bottom_right_y) / 2
        df_temp['bbox_center_x_norm'] = (bbox_top_left_x + bbox_bottom_right_x) / 2 / df_temp['imageWidth']
        df_temp['bbox_center_y_norm'] = (bbox_top_left_y + bbox_bottom_right_y) / 2 / df_temp['imageHeight']
        
    if each_ann['shape_type'] == 'point': # 筛选出关键点标注
        
        # 该点的 XY 坐标
        kpt_xy = each_ann['points'][0]
        kpt_x, kpt_y = int(kpt_xy[0]), int(kpt_xy[1])
        
        # 标注信息
        df_temp['label_type'] = each_ann['shape_type']
        df_temp['label'] = each_ann['label'] 
        
        # 关键点坐标信息
        df_temp['kpt_x_pix'] = kpt_x
        df_temp['kpt_y_pix'] = kpt_y
        df_temp['kpt_x_norm'] = kpt_x / df_temp['imageWidth']
        df_temp['kpt_y_norm'] = kpt_y / df_temp['imageHeight']
        
    if each_ann['shape_type'] == 'polygon': # 筛选出多段线（polygon）标注
        
        poly_points = np.array(each_ann['points']).astype('uint32').tolist() # 该多段线每个点的坐标
        poly_num_points = len(poly_points) # 该多段线点的个数
        
        # 计算多段线区域面积
        poly_pts = [np.array(each_ann['points'], np.int32).reshape((-1, 1, 2))] # 该多段线每个点的坐标
        img_bgr = cv2.imread(imagePath)
        img_zeros = np.zeros(img_bgr.shape[:2], dtype='uint8')
        img_mask = cv2.fillPoly(img_zeros, poly_pts, 1)
        poly_area = np.sum(img_mask)
        
        # 标注信息
        df_temp['label_type'] = each_ann['shape_type']
        df_temp['label'] = each_ann['label']
        
        # 多段线信息
        df_temp['poly_points'] = poly_points
        df_temp['poly_num_points'] = poly_num_points
        df_temp['poly_area'] = poly_area
        
        
    df = df.append(df_temp, ignore_index=True)

100%|██████████| 10/10 [00:00<00:00, 24.74it/s]


In [60]:
df.columns

Index(['imagePath', 'imageWidth', 'imageHeight', 'label_type', 'label',
       'bbox_top_left_x', 'bbox_top_left_y', 'bbox_bottom_right_x',
       'bbox_bottom_right_y', 'bbox_width_pix', 'bbox_height_pix',
       'bbox_width_norm', 'bbox_height_norm', 'bbox_center_x_pix',
       'bbox_center_y_pix', 'bbox_center_x_norm', 'bbox_center_y_norm',
       'kpt_x_norm', 'kpt_x_pix', 'kpt_y_norm', 'kpt_y_pix', 'poly_area',
       'poly_num_points', 'poly_points'],
      dtype='object')

In [61]:
df

,imagePath,imageWidth,imageHeight,label_type,label,bbox_top_left_x,bbox_top_left_y,bbox_bottom_right_x,bbox_bottom_right_y,bbox_width_pix,...,bbox_center_y_pix,bbox_center_x_norm,bbox_center_y_norm,kpt_x_norm,kpt_x_pix,kpt_y_norm,kpt_y_pix,poly_area,poly_num_points,poly_points
0,1.jpg,3648.0,2736.0,rectangle,sjb_rect,132.0,829.0,2209.0,1818.0,2077.0,...,1323.5,0.320861,0.483735,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1.jpg,3648.0,2736.0,rectangle,sjb_rect,2368.0,429.0,3524.0,2273.0,1156.0,...,1351.0,0.807566,0.493787,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1.jpg,3648.0,2736.0,point,angle_30,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.041941,153.0,0.315058,862.0,NaN,NaN,NaN
3,1.jpg,3648.0,2736.0,point,angle_30,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.655428,2391.0,0.164474,450.0,NaN,NaN,NaN
4,1.jpg,3648.0,2736.0,point,angle_60,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.601425,2194.0,0.327851,897.0,NaN,NaN,NaN
5,1.jpg,3648.0,2736.0,point,angle_60,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.959430,3500.0,0.810673,2218.0,NaN,NaN,NaN
6,1.jpg,3648.0,2736.0,point,angle_90,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.456414,1665.0,0.656798,1797.0,NaN,NaN,NaN
7,1.jpg,3648.0,2736.0,point,angle_90,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.665844,2429.0,0.815789,2232.0,NaN,NaN,NaN
8,1.jpg,3648.0,2736.0,polygon,sjb_poly,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,957783.0,3.0,"[[135, 862], [2203, 897], [1665, 1812]]"
9,1.jpg,3648.0,2736.0,polygon,sjb_poly,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,987034.0,3.0,"[[2391, 438], [2424, 2238], [3518, 2232]]"
